In [1]:
import pandas as pd
import numpy as np
from os.path import join
from pathlib import Path
import os


In [2]:
path = Path("data/")
filename_1 = 'train_dataset.csv'
filename_2 = 'preprocessing_loaded_table.csv'




In [3]:
#Функция для возврата часа в дату
def date_plus_hours_to_index(data):
    data = data.copy()
    data['date'] = data['date'] + pd.to_timedelta(data['time'] , 'H')
    data = data.set_index('date')
    return data

In [4]:
# Функция для сдвига на суткм (в скаченном датасете разбивка по 30 мин, по этому timeshift=48)
def shift_features_fact(df, timeshift=48):
    list_fact_columns=list([df.columns])
    new_df = df.copy()
    for column in list_fact_columns:
        new_df[column] = new_df[column].shift(timeshift)
    
    return new_df

In [5]:
# объединение оригинальных тренировочных данных и скаченных признаков со сдвигом
def data_merge(path, filename_1, filename_2):
    df_train = pd.read_csv(join(path, filename_1), parse_dates=[0])
    df_train = date_plus_hours_to_index(df_train)
    df_fich = pd.read_csv(join(path, filename_2))
    df_fich['date'] = pd.to_datetime(df_fich['date'], infer_datetime_format=True)
    df_fich = df_fich.set_index('date')
    df_fich_shift = shift_features_fact(df_fich)

    
    
    df_train = pd.merge(df_train, df_fich_shift[df_fich.columns], left_index=True, right_index=True)

    return df_train

    
    

In [6]:
df_train_shift = data_merge(path, 'train_dataset.csv', 'preprocessing_loaded_table.csv')


In [7]:
df_train_shift.to_csv('data/df_train_shift.csv')